# Titanic: Machine Learning from Disaster (More) 

1차 solution : https://github.com/yudong80/dastudy/blob/master/datalabs/01.titanic/yudong_titanic.ipynb      

앞의 논의에 이어 좀더 재밌게 데이터를 분석해봅시다 
또다시 질문기반으로 접근해봅니다 

## 질문1: train과 test는 유사한가? 

이 질문이 중요한 이유는 제가 본 솔루션의 근본적인 가정은 train 으로 훈련을 한 후에 그것의 점수를 바탕으로 test 데이터에서도 높은 점수를 얻을 수 있다고 가정하고 있기 때문입니다. 

그런데 만약 train과 test의 성질이 같지 않다면 혹은 크게 다르다면 그러한 가정이 깨지는 것이죠. 
그렇게 되면 train에서 높은 점수를 얻었다고 하더라도 test에서 같은 점수가 나온다는 보장이 없습니다. 

## 1.1 모두 생존 가정을 train에 적용해보기 

그러면 몇점이나 나올까요? test의 경우 (0.3732) 가 나왔습니다. 

In [1]:
## 공통부분 
import pandas as pd

train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

In [2]:
# 모두 생존 가정 
cpy = train.copy() 
cpy["Survived2"] = 1 

In [3]:
cpy.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived2
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1


In [13]:
cpy["Chk"] = cpy["Survived"] == 1 
chk_mapping = {True: 1, False: 0} 
cpy["Chk"] = cpy["Chk"].map(chrk_mapping)

In [14]:
cpy.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived2,Chk
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,0


### 확인결과 (test, train) = (0.3732, 0.3838) : 비슷하다고 할 수 있나요?  

#### test 대비 1.02로 2% 밖에 차이가 안납니다. 

In [20]:
#점수확인  
test_point = 0.3732 
train_point = cpy.Chk.sum() / 891
print("train point : ", train_point , " & 유사도 : " , train_point / test_point) 

train point :  0.3838383838383838  & 유사도 :  1.028505851656977


## 1.2 모두 사망 가정은 어떨까? (test: 0.62679)

#### test 대비 0.98로 2% 밖에 차이가 안납니다.

마치 사전에 두개의 set가 유사하도록 맞춰 놓은 것 같습니다. 

In [23]:
# 모두 사망 가정 
cpy = train.copy() 
cpy["Survived2"] = 0

# Chk 컬럼 생성 
cpy["Chk"] = cpy["Survived"] == 0 
chk_mapping = {True: 1, False: 0} 
cpy["Chk"] = cpy["Chk"].map(chk_mapping)

#점수확인  
test_point = 0.62679 
train_point = cpy.Chk.sum() / 891
print("train point : ", train_point , " & 유사도 : " , train_point / test_point) 

train point :  0.6161616161616161  & 유사도 :  0.9830431502761948


## 1.3 Age, Pclass, Sex의 경우는 어떨까? 

In [24]:
# Feature Engineering 

train_test_data = [train, test] # combining train and test dataset

for dataset in train_test_data:
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    
title_mapping = {"Mr": 0, "Miss": 1, "Mrs": 2, 
                 "Master": 3, "Dr": 3, "Rev": 3, "Col": 3, "Major": 3, "Mlle": 3,"Countess": 3,
                 "Ms": 3, "Lady": 3, "Jonkheer": 3, "Don": 3, "Dona" : 3, "Mme": 3,"Capt": 3,"Sir": 3 }
for dataset in train_test_data:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    
# fill missing age with median age for each title (Mr, Mrs, Miss, Others)
train["Age"].fillna(train.groupby("Title")["Age"].transform("median"), inplace=True)
test["Age"].fillna(test.groupby("Title")["Age"].transform("median"), inplace=True)

for dataset in train_test_data:
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0,
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 26), 'Age'] = 1,
    dataset.loc[(dataset['Age'] > 26) & (dataset['Age'] <= 36), 'Age'] = 2,
    dataset.loc[(dataset['Age'] > 36) & (dataset['Age'] <= 62), 'Age'] = 3,
    dataset.loc[ dataset['Age'] > 62, 'Age'] = 4
    
sex_mapping = {"male": 0, "female": 1}
for dataset in train_test_data:
    dataset['Sex'] = dataset['Sex'].map(sex_mapping)    

In [26]:
#Machine Learning 
cpy = pd.DataFrame({
        "Pclass": train["Pclass"],
        "Sex": train["Sex"],
        "Age": train["Age"]
    }).copy()

# Importing Classifier Modules
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

import numpy as np

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

train_data = cpy
target = train['Survived'] 
clf = SVC()
clf.fit(train_data, target)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

여기까지 하다보니.. 머가 이상한 것이 모델 fit을 train 으로 하였는데 , 
그것으로 train의 실제 Survived 컬럼과 비교하려고 하니.. 

#### 당연히 비슷할 것이라는 생각이 들었습니다. 

그래서 (허민석님)의 solution에서는 k-fold 기법을 이용한 Cross Validation을 한 것 같습니다. 

#### 이제야 왜 cross_val_score() 함수를 썼는지 이해가 되었네요. 
#### 그러면 여기에서도 한번 해볼까요? 

## 2. Cross Validation 적용해보기 

SVM 기반 + Age, Pclass, Sex 컬럼 활용한 Cross Validation 결과는 

허민석님의 경우 (83.5점) / 제 경우 (79.24점) 이 나왔네요. 


실제 test에 넣었을 때의 점수의 경우

허민석님 (78.94점) / 제 경우 (77.51점) 

#### 최종 점수는 1.4점 정도 차이나는데 등수차이가 많이 나네요 :-) 

최종 등수는  
#### 허민석님 (3623등) / 제 경우 (6373등) 

여기까지입니다. 
허민석님의 submission 파일은 submission_heo.csv 입니다. 

In [33]:
clf = SVC()
scoring = 'accuracy'
score = cross_val_score(clf, train_data, target, cv=k_fold, n_jobs=1, scoring=scoring)
print("score : " , score)
print("score average", round(np.mean(score)*100,2))

score :  [0.77777778 0.83146067 0.79775281 0.7752809  0.83146067 0.79775281
 0.82022472 0.78651685 0.78651685 0.71910112]
score average 79.24
